Récupération et exploration du corpus d'un texte
--------------------------

Il est important de bien identifier 4 termes incontournables : 

- le corpus : un ensemble de documents (des textes dans notre cas), regroupés dans une optique ou dans une thématique précise. 

- un document : la notion de document fait référence à un texte appartenant au corpus, mais indépendant des autres textes. Il peut être constitué d'une ou plusieurs phrases, un ou plusieurs paragraphes.

- un token : le terme token désigne généralement un mot et/ou un élément de ponctuation. La phrase "Hello World!" comprend donc 3 tokens. 

- le vocabulaire : il s'agit de l'ensemble des tokens distincts présents dans l'ensemble du corpus.

Pour les documents .txt, .doc, .csv, .xls, cela semble facile, mais comment faire pour les documents .pdf ?  

Transformer les documents .pdf en .txt n'est pas une chose facile. Ce traitement spécifique à un nom, cela s'appelle OCR :  Optical Character Recognition. 

Heureusement, il existe de nombreux packages disponibles, comme par exemple Tesseract.

Nous allons aborder trois pré-traitements à effectuer:
- Récupération et traitement du corpus
- La tokénisation du corpus
- La normalisation et la construction du dictionnaire

Le cycle de traitement d'un corpus

![pipeline](pipeline1.png)

### Récupération du corpus de texte

Attention au format d'encodage de vos fichiers texte qui peuvent mener à des erreurs faciles à éviter. On favorisera l'UTF-8 (encodage universel) très utilisé et qui permet l'usage des accents.

![corpus_structure](text-corpus-structure.png)

In [1]:
import nltk

In [10]:
nltk.corpus.brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [11]:
# prenons science_fiction et humor
words = nltk.corpus.brown.words(categories=['humor', 'science_fiction'])

In [12]:
len(words)

36165

Scraping des descriptions des artistes francais disponible sur wikipedia https://fr.wikipedia.org/wiki/Liste_d%27artistes_fran%C3%A7ais

In [35]:
import requests as rq
from bs4 import BeautifulSoup
from tqdm import tqdm

In [46]:
url = r"https://fr.wikipedia.org/wiki/Liste_d%27artistes_fran%C3%A7ais"

with rq.get(url) as response:
    if response.ok:
        text = response.text
        
        artistes = {}
        
        try:
            dom = BeautifulSoup(text, "lxml")
            artistes_li = dom.select('div[id = "mw-content-text"] li a[href *= "/wiki"]')
            
            # après récupération des listes nous pouvons passer à celles des noms et des descriptions
            
            for artiste in tqdm(artistes_li):
                # recuperation du nom
                name = artiste.text
                
                # recuperation de la description a partir du lien wikipedia
                link = artiste.get('href')
                
                full_link = "".join(["https://fr.wikipedia.org/", link])
                
                descriptions = []
                
                with rq.get(full_link) as response2:
                    
                    if response2.ok:
                        
                        text2 = response2.text
                    
                        try:
                            dom2 = BeautifulSoup(text2, "lxml")

                            paragraphs = dom.select('p')
                        
                            for paragraph in paragraphs:
                                descriptions.append(paragraph.text)
                            
                        except Exception as e:
                            print(f"Error part 2 {e}")
                
                if len(descriptions) != 0:
                     artistes[name] = "\n".join(descriptions)
                
        except Exception as e:
            print(f"Error {e}")

 75%|███████▍  | 299/399 [10:29<03:34,  2.14s/it]

In [41]:
artistes

{}

### Exploration du text: tokenisation et analyse des fréquences

In [37]:
text = "Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !"

nltk.word_tokenize(text)

['Bonjour',
 ',',
 'je',
 'suis',
 'un',
 'texte',
 "d'exemple",
 'pour',
 'le',
 'cours',
 "d'Openclassrooms",
 '.',
 'Soyez',
 'attentifs',
 'à',
 'ce',
 'cours',
 '!']

Les accents sont affichés encodés mais ce n'est pas très grave, on peut revenir à un affichage normal si on le souhaite.

Le fait d'essayer d'harmoniser les tokens est un processus nommé « normalisation ». 

https://www.debuggex.com/cheatsheet/regex/python

In [39]:
tokenizer = nltk.RegexpTokenizer(r'\w+')

tokenizer.tokenize(text.decode('utf-8').lower())

AttributeError: 'str' object has no attribute 'decode'